<p><font size="6" color='grey'> <b>

Generative KI. Verstehen. Anwenden. Gestalten.
</b></font> </br></p>

<p><font size="5" color='grey'> <b> Chat Memory Patterns</b></font> </br></p>

---

In [ ]:
#@title 🔧 Umgebung einrichten{ display-mode: "form" }
!uv pip install --system -q git+https://github.com/ralf-42/GenAI.git#subdirectory=04_modul
from genai_lib.utilities import (
    check_environment,
    get_ipinfo,
    setup_api_keys,
    mprint,
    install_packages,
    mermaid,
    get_model_profile,
    extract_thinking,
    load_chat_prompt_template
)
setup_api_keys(['OPENAI_API_KEY'], create_globals=False)
print()
check_environment()
print()
get_ipinfo()

<p><font color='black' size="5">
⏸️ 5-Minuten-Check:
</font></p>

**Ziel:** Prüfen, ob du das vorherige Kapitel verstanden hast – nicht, ob es gerade läuft.

**Aufgabe** (5 Minuten, ohne Vorlage):

Rekonstruiere die zentrale Idee oder Code-Struktur des letzten Abschnitts selbstständig
(kein Copy & Paste, kein Nachschlagen).

Wähle eine der folgenden Optionen:

+ Erkläre in 1–2 Sätzen, was hier konzeptionell passiert.

+ Verändere eine Kleinigkeit (z. B. Prompt, Parameter, Reihenfolge) und beschreibe die Auswirkung.

+ Markiere eine Stelle, die du nicht sicher erklären kannst, und formuliere eine konkrete Frage dazu.

**Hinweis:**
Nicht alles muss „fertig“ oder „korrekt“ sein. Entscheidend ist, wo dein Verständnis gerade endet

# 1 | Intro
---

<p><font color='black' size="5">
Zustandslosigkeit von LLMs
</font></p>

Large Language Models (LLMs) wie GPT sind von Natur aus **zustandslos** - sie verfugen uber kein eingebautes Gedachtnis. Jede Anfrage wird isoliert verarbeitet, ohne Bezug zu vorherigen Interaktionen. Deshalb muss der Chatverlauf (Historie) bei jeder Anfrage neu ubergeben werden.

```
Ohne Memory:
User: "Mein Name ist Max"
AI: "Hallo Max!"
User: "Wie heisse ich?"
AI: "Das habe ich nicht gespeichert."
```

**Wir verwenden RunnableWithMessageHistory** – den etablierten LangChain-Ansatz für Konversations-Memory. Für komplexere Anwendungen mit mehreren Agenten oder fortgeschrittener Zustandsverwaltung bietet **LangGraph** erweiterte Möglichkeiten.

**Dieses Notebook zeigt Memory-Patterns mit RunnableWithMessageHistory:**

| Pattern | Beschreibung | Anwendungsfall |
|---------|--------------|----------------|
| **Python-Liste** | Einfachste Losung | Prototyping, kurze Sessions |
| **Trimming** | Nur letzte N Nachrichten | Token-Limit einhalten |
| **Summary** | Alte Nachrichten zusammenfassen | Lange Sessions, Kontext erhalten |
| **FileChatMessageHistory** | Persistenz in Dateien | Einfache Persistenz ohne DB |
| **Datenbank** | Persistente Speicherung | Production, Multi-User |

In [ ]:
#@title 🔧 🧜‍♀️Mermaid-Diagramm { display-mode: "form" }

diagram = """
graph TD
    root[Chat Memory Patterns]

    %% 1. Manuelle Variante
    root -->|1 Short-term / Manuell| manual[Python-Liste]
    manual --> manual_desc["<b>Beschreibung:</b> Einfache Liste im RAM<br/><b>Use Case:</b> Prototyping, kurze Sessions"]

    %% 2. LCEL Variante
    root -->|2 Automatisch / LCEL| lcel[RunnableWithMessageHistory]

    %% Optimierungs-Strategien
    lcel -->|Context Management| strat[Strategien mit Token-Limits]

    strat --> trim[Trimming / Sliding Window]
    trim --> trim_desc["<b>Funktion:</b> Nur letzte N Nachrichten behalten<br/><b>Ziel:</b> Token-Limit einhalten"]

    strat --> sum[Summary]
    sum --> sum_desc["<b>Funktion:</b> Alte Nachrichten per LLM zusammenfassen<br/><b>Ziel:</b> Langzeit-Kontext erhalten"]

    %% Persistenz-Lösungen
    lcel -->|Long-term Memory| persist[Persistenz-Backends]

    persist --> file[FileChatMessageHistory]
    file --> file_desc["<b>Speicher:</b> JSON-Dateien<br/><b>Use Case:</b> Einfache Persistenz ohne DB"]

    persist --> db[Datenbank / SQLite]
    db --> db_desc["<b>Speicher:</b> Relationale Datenbank<br/><b>Use Case:</b> Production, Multi-User"]

    %% Styling
    style root fill:#2ecc71,stroke:#27ae60,stroke-width:2px,color:white
    style lcel fill:#3498db,stroke:#2980b9,stroke-width:2px,color:white
    style manual fill:#95a5a6,stroke:#7f8c8d,stroke-width:2px,color:white
    style strat fill:#f1c40f,stroke:#f39c12,color:black
    style persist fill:#e67e22,stroke:#d35400,color:white
"""
mermaid(diagram, width=1000, height=800)

# 2 | Short-term Memory (Python-Liste)
---

Die einfachste Form von Memory: Eine **Python-Liste**, die alle Nachrichten speichert und bei jedem API-Call mitgesendet wird.

In [ ]:
# Importe
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser

# System-Prompt
system_prompt = "Du bist ein hilfreicher und humorvoller KI-Assistent."

# Prompt-Template mit Historie (MessagesPlaceholder nimmt die Historie entgegen)
prompt = ChatPromptTemplate.from_messages([
    ("system", "{system_prompt}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{user_input}")
])

# LLM (Kurznotation: "provider:model")
llm = init_chat_model("openai:gpt-4o-mini", temperature=0)

# Parser
parser = StrOutputParser()

# Chain
chain = prompt | llm | parser

In [ ]:
# Chat-Funktion mit manueller Historien-Verwaltung
def chat(chat_history, user_input):
    """Fuhrt eine Chat-Interaktion mit manueller Historien-Verwaltung durch."""

    # Chain aufrufen (Historie wird im Prompt mitgeschickt)
    response = chain.invoke({
        'system_prompt': system_prompt,
        'chat_history': chat_history,
        'user_input': user_input
    })

    # Ausgabe
    mprint(f"### 🧑‍🦱 Mensch:\n{user_input}")
    mprint(f"### 🤖 KI:\n{response}\n")

    # Memory (Liste) MANUELL aktualisieren
    chat_history.append(HumanMessage(content=user_input))
    chat_history.append(AIMessage(content=response))

    return chat_history

In [5]:
# Historie initialisieren
chat_history = [SystemMessage(content=system_prompt)]


# Konversation
chat_history = chat(chat_history, "Mein Name ist Max")
chat_history = chat(chat_history, "Ich mag Python-Programmierung")
chat_history = chat(chat_history, "Weisst du noch, wie ich heisse und was ich mag?")

mprint("### Gespeicherte Nachrichten (Liste):\n---")
for msg in chat_history:
    mprint(f"  **{msg.type}**:   {msg.content}")

### 🧑‍🦱 Mensch:
Ich mag Python-Programmierung

### 🤖 KI:
Das ist großartig, Max! Python ist eine fantastische Sprache – so vielseitig und benutzerfreundlich. Was machst du gerne mit Python? Programmierst du Spiele, Webanwendungen oder vielleicht etwas ganz anderes?


### 🧑‍🦱 Mensch:
Weisst du noch, wie ich heisse und was ich mag?

### 🤖 KI:
Natürlich, Max! Du magst Python-Programmierung. Ich bin wie ein Elefant – ich vergesse nie! Gibt es etwas Bestimmtes, worüber du in Bezug auf Python sprechen möchtest? Vielleicht ein Projekt oder eine Herausforderung?


### Gespeicherte Nachrichten (Liste):
---

...

  **ai**:   Hallo Max! Schön, dich kennenzulernen! Wie kann ich dir heute helfen? Oder möchtest du einfach ein bisschen plaudern?

  **human**:   Ich mag Python-Programmierung

  **ai**:   Das ist großartig, Max! Python ist eine fantastische Sprache – so vielseitig und benutzerfreundlich. Was machst du gerne mit Python? Programmierst du Spiele, Webanwendungen oder vielleicht etwas ganz anderes?

  **human**:   Weisst du noch, wie ich heisse und was ich mag?

  **ai**:   Natürlich, Max! Du magst Python-Programmierung. Ich bin wie ein Elefant – ich vergesse nie! Gibt es etwas Bestimmtes, worüber du in Bezug auf Python sprechen möchtest? Vielleicht ein Projekt oder eine Herausforderung?

<p><font color='darkblue' size="4">
Problem:
</font></p>

- Keine automatische Session-Verwaltung (Multi-User)
- Manuelles Memory-Management fehleranfallig
- **Bei langen Konversationen: Token-Limit wird uberschritten!**

# 3 | RunnableWithMessageHistory (LCEL)
---

`RunnableWithMessageHistory` ist die **offizielle LCEL-Alternative** zu den deprecated Memory-Klassen (`ConversationBufferMemory`, etc.). Es *umschließt* eine Chain und übernimmt die automatische Verwaltung der Chat-Historie.

**Vorteile gegenuber manueller Verwaltung:**
- Automatisches Laden/Speichern der Historie
- Session-Management uber `session_id`
- Kompatibel mit Streaming und Async

```python
from langchain_core.runnables.history import RunnableWithMessageHistory
```

## 3.1 | Basismodell

In [6]:
# Importe fur RunnableWithMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory

# Session-Store (Dictionary: session_id -> ChatMessageHistory)
session_store = {}

def get_session_history(session_id):
    """Gibt die Historie fur eine Session zuruck (oder erstellt eine neue)."""
    if session_id not in session_store:
        session_store[session_id] = InMemoryChatMessageHistory()
    return session_store[session_id]

In [7]:
# Prompt mit Historie-Platzhalter
prompt_with_history = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    MessagesPlaceholder(variable_name="history"),  # Hier wird die Historie eingefugt
    ("human", "{user_input}")
])

# Basis-Chain (ohne Memory)
base_chain = prompt_with_history | llm | parser

# Chain MIT automatischem Memory-Management
chain_with_history = RunnableWithMessageHistory(
    runnable=base_chain,
    get_session_history=get_session_history,
    input_messages_key="user_input",        # Key fur neue User-Nachricht
    history_messages_key="history"          # Key fur die Historie im Prompt
)

print("Chain mit RunnableWithMessageHistory erstellt")

Chain mit RunnableWithMessageHistory erstellt


In [8]:
# Helper-Funktion fur Chat mit Session-ID
def chat_with_session(session_id, user_input):
    """Chattet mit automatischem Memory uber RunnableWithMessageHistory."""

    # Config mit Session-ID (PFLICHT!)
    config = {"configurable": {"session_id": session_id}}

    # Chain aufrufen - Historie wird automatisch geladen/gespeichert
    response = chain_with_history.invoke(
        {"user_input": user_input},
        config=config
    )

    mprint(f"**[{session_id}] 🧑‍🦱 Mensch:** {user_input}")
    mprint(f"**[{session_id}] 🤖 KI:** {response}\n")

    return response

In [ ]:
# Demo: RunnableWithMessageHistory
mprint("## RunnableWithMessageHistory Demo")
mprint("---")

# Session 1: Max
chat_with_session("max", "Hallo! Ich bin Max aus München.")
chat_with_session("max", "Ich programmiere gerne in Python.")

# Session 2: Emma (separate Historie!)
chat_with_session("emma", "Hi! Ich bin Emma und mag Machine Learning.")

# Zuruck zu Max - Memory bleibt erhalten!
chat_with_session("max", "Woher komme ich und was ist mein Hobby?")

# Session-Store anzeigen
mprint("### Gespeicherte Sessions:")
for sid, history in session_store.items():
    mprint(f"- **{sid}**: {len(history.messages)} Nachrichten")

## RunnableWithMessageHistory Demo

---

In [ ]:
# Historie einer Session anzeigen
def show_session_history(session_id):
    """Zeigt die Nachrichten einer Session."""
    if session_id not in session_store:
        print(f"Session '{session_id}' nicht gefunden")
        return

    history = session_store[session_id]
    mprint(f"### Historie: {session_id}")
    for i, msg in enumerate(history.messages, 1):
        role = "Human" if msg.type == "human" else "KI"
        mprint(f"{i}. **{role}:** {msg.content}")

show_session_history("max")

## 3.2 | Trimming (Sliding Window)

**Problem:** Bei langen Konversationen wächst die Historie unbegrenzt – Token-Limit wird überschritten!

**Lösung:** **Trimming** (Sliding Window) – Nur die letzten N Nachrichten werden behalten.

```python
from langchain_core.messages import trim_messages

# Automatisches Trimming: Nur letzte 6 Nachrichten
trimmer = RunnableLambda(trim_messages(max_tokens=6, strategy="last"))
chain_with_trimming = trimmer | chain_with_history
```

In [ ]:
# Trimming-Implementierung: Nur letzte N Nachrichten behalten
from langchain_core.messages import trim_messages
from langchain_core.runnables import RunnableLambda

MAX_MESSAGES = 6  # Maximal 6 Nachrichten im Kontext (3 Runden)

# Trimmer-Funktion, die das gesamte Input-Dict verarbeitet
def apply_trimming(input_dict):
    """
    Verarbeitet das Input-Dictionary und trimmt die Historie.
    RunnableWithMessageHistory gibt {"user_input": ..., "history": [...]} weiter.
    """
    history = input_dict.get("history", [])

    # Trimming: Nur letzte N Nachrichten behalten
    trimmed_history = trim_messages(
        history,
        max_tokens=MAX_MESSAGES,
        strategy="last",
        token_counter=len  # Zähle Nachrichten, nicht Tokens
    )

    # Neues Dictionary mit getrimmter Historie zurückgeben
    return {
        "user_input": input_dict["user_input"],
        "history": trimmed_history
    }

# Chain mit Trimming
trimmed_chain = (
    RunnableLambda(apply_trimming)
    | prompt_with_history
    | llm
    | parser
)

# Mit RunnableWithMessageHistory wrappen
chain_with_trimming = RunnableWithMessageHistory(
    runnable=trimmed_chain,
    get_session_history=get_session_history,
    input_messages_key="user_input",
    history_messages_key="history"
)

print(f"Chain mit Trimming erstellt (max. {MAX_MESSAGES} Nachrichten)")

In [ ]:
# Demo: Trimming in Aktion
mprint("## Trimming Demo (max. 6 Nachrichten = 3 Runden)")
mprint("---")

config_trim = {"configurable": {"session_id": "trim_test"}}

# Runde 1: Info 1
response = chain_with_trimming.invoke({"user_input": "Mein Name ist Max."}, config=config_trim)
mprint(f"**1. Mensch:** Mein Name ist Max.")
mprint(f"**1. KI:** {response}\n")

# Runde 2: Info 2
response = chain_with_trimming.invoke({"user_input": "Ich komme aus München."}, config=config_trim)
mprint(f"**2. Mensch:** Ich komme aus München.")
mprint(f"**2. KI:** {response}\n")

# Runde 3: Info 3
response = chain_with_trimming.invoke({"user_input": "Ich mag Python."}, config=config_trim)
mprint(f"**3. Mensch:** Ich mag Python.")
mprint(f"**3. KI:** {response}\n")

# Runde 4: Info 4 (Jetzt sollte Runde 1 vergessen sein!)
response = chain_with_trimming.invoke({"user_input": "Ich arbeite als Data Scientist."}, config=config_trim)
mprint(f"**4. Mensch:** Ich arbeite als Data Scientist.")
mprint(f"**4. KI:** {response}\n")

# Test: Erinnert sich der Bot an Info 1? (sollte NEIN sein)
response = chain_with_trimming.invoke({"user_input": "Wie heisse ich?"}, config=config_trim)
mprint(f"**5. Mensch:** Wie heisse ich?")
mprint(f"**5. KI:** {response}")
mprint("\n⚠️ **Ergebnis:** Der Bot erinnert sich NICHT mehr an 'Max' (wurde getrimmt!)")

## 3.3 | Summary (LLM-basierte Zusammenfassung)

**Problem:** Trimming verwirft alte Informationen vollständig – wichtiger Kontext geht verloren.

**Alternative:** **Summarization** – Ein LLM fasst alte Nachrichten zusammen, die Zusammenfassung wird als Kontext verwendet.

**Strategie:**
1. Wenn Historie > Limit: Alte Nachrichten zusammenfassen
2. Summary als Teil des System-Prompts einfügen
3. Nur letzte N Nachrichten + Summary behalten

```python
# Pseudo-Code
if len(history) > MAX_MESSAGES:
    summary = llm.invoke("Fasse zusammen: {old_messages}")
    new_history = [summary] + recent_messages
```

In [ ]:
# Summary-Implementierung
SUMMARY_THRESHOLD = 8  # Ab 8 Nachrichten: Summarize
KEEP_RECENT = 4        # Behalte die letzten 4 Nachrichten

def summarize_messages(messages):
    """
    Fasst alte Nachrichten zusammen, wenn das Limit überschritten wird.
    Behält nur Summary + letzte N Nachrichten.
    """
    if len(messages) <= SUMMARY_THRESHOLD:
        return messages  # Noch unter Limit, nichts tun

    # Alte Nachrichten (zum Zusammenfassen)
    old_messages = messages[:-KEEP_RECENT]
    recent_messages = messages[-KEEP_RECENT:]

    # Summary erstellen (LLM-basiert)
    summary_prompt = ChatPromptTemplate.from_messages([
        ("system", "Fasse die folgende Konversation prägnant zusammen. Extrahiere wichtige Fakten (Namen, Orte, Präferenzen)."),
        ("human", "{conversation}")
    ])

    # Konversation als Text
    conversation_text = "\n".join([
        f"{msg.type}: {msg.content}" for msg in old_messages
    ])

    # Summary generieren
    summary_chain = summary_prompt | llm | parser
    summary_text = summary_chain.invoke({"conversation": conversation_text})

    # Summary als SystemMessage
    summary_message = SystemMessage(content=f"Zusammenfassung der bisherigen Konversation:\n{summary_text}")

    # Neue Historie: Summary + letzte N Nachrichten
    return [summary_message] + recent_messages

# Chain mit Summary
summary_chain = (
    RunnableLambda(summarize_messages)
    | prompt_with_history
    | llm
    | parser
)

# Mit RunnableWithMessageHistory wrappen
chain_with_summary = RunnableWithMessageHistory(
    runnable=summary_chain,
    get_session_history=get_session_history,
    input_messages_key="user_input",
    history_messages_key="history"
)

print(f"Chain mit Summary erstellt (Limit: {SUMMARY_THRESHOLD}, behalte letzte {KEEP_RECENT})")

In [ ]:
# Demo: Summary in Aktion
mprint("## Summary Demo (Limit: 8 Nachrichten, behalte 4)")
mprint("---")

config_summary = {"configurable": {"session_id": "summary_test"}}

# Mehrere Runden simulieren (um Limit zu überschreiten)
conversations = [
    "Mein Name ist Max.",
    "Ich bin 30 Jahre alt.",
    "Ich komme aus München.",
    "Ich arbeite als Data Scientist.",
    "Mein Hobby ist Fotografie.",
    "Ich mag italienisches Essen."  # Nach dieser Nachricht: 12 Messages (6 Runden) -> Summary!
]

for i, user_msg in enumerate(conversations, 1):
    response = chain_with_summary.invoke({"user_input": user_msg}, config=config_summary)
    mprint(f"**{i}. Mensch:** {user_msg}")
    mprint(f"**{i}. KI:** {response}\n")

# Test: Erinnert sich der Bot an alte Infos? (sollte JA sein, via Summary!)
response = chain_with_summary.invoke(
    {"user_input": "Fasse zusammen: Wie heisse ich, wie alt bin ich, woher komme ich?"},
    config=config_summary
)
mprint(f"**Test. Mensch:** Fasse zusammen: Wie heisse ich, wie alt bin ich, woher komme ich?")
mprint(f"**Test. KI:** {response}")
mprint("\n✅ **Ergebnis:** Der Bot erinnert sich trotz Summary an alte Infos!")

# 4 | Long-term Memory
---

## 4.1 | FileChatMessageHistory (Persistenz in Dateien)

**Problem:** `InMemoryChatMessageHistory` verliert die Historie beim Neustart der Anwendung.

**Lösung:** **FileChatMessageHistory** – Speichert jede Session in einer separaten JSON-Datei.

**Vorteile:**
- Einfache Persistenz ohne Datenbank
- Menschenlesbare JSON-Files
- Ideal für Prototyping und lokale Apps

```python
from langchain_community.chat_message_histories import FileChatMessageHistory

# Jede session_id bekommt eine eigene Datei
history = FileChatMessageHistory(file_path="./chat_sessions/session_123.json")
```

In [ ]:
# FileChatMessageHistory Setup
import os
from langchain_community.chat_message_histories import FileChatMessageHistory

# Verzeichnis für Session-Files erstellen
SESSION_DIR = "./chat_sessions"
os.makedirs(SESSION_DIR, exist_ok=True)

def get_file_session_history(session_id: str):
    """
    Gibt eine FileChatMessageHistory für die Session zurück.
    Jede Session wird in einer separaten JSON-Datei gespeichert.
    """
    file_path = os.path.join(SESSION_DIR, f"{session_id}.json")
    return FileChatMessageHistory(file_path=file_path)

# Chain mit File-basiertem Memory
chain_with_file_history = RunnableWithMessageHistory(
    runnable=base_chain,
    get_session_history=get_file_session_history,
    input_messages_key="user_input",
    history_messages_key="history"
)

print(f"Chain mit FileChatMessageHistory erstellt (Speicherort: {SESSION_DIR})")

In [ ]:
# Demo: FileChatMessageHistory
mprint("## FileChatMessageHistory Demo")
mprint("---")

config_file = {"configurable": {"session_id": "max_file"}}

# Konversation führen (wird automatisch in Datei gespeichert)
response = chain_with_file_history.invoke(
    {"user_input": "Hallo! Ich bin Max aus München."},
    config=config_file
)
mprint(f"**Mensch:** Hallo! Ich bin Max aus München.")
mprint(f"**KI:** {response}\n")

response = chain_with_file_history.invoke(
    {"user_input": "Ich mag Python-Programmierung."},
    config=config_file
)
mprint(f"**Mensch:** Ich mag Python-Programmierung.")
mprint(f"**KI:** {response}\n")

# Test: Memory-Recall
response = chain_with_file_history.invoke(
    {"user_input": "Woher komme ich und was mag ich?"},
    config=config_file
)
mprint(f"**Mensch:** Woher komme ich und was mag ich?")
mprint(f"**KI:** {response}")

In [ ]:
# Gespeicherte Session-Files anzeigen
mprint("### Gespeicherte Session-Dateien:")
mprint("---")

for filename in os.listdir(SESSION_DIR):
    if filename.endswith(".json"):
        filepath = os.path.join(SESSION_DIR, filename)
        file_size = os.path.getsize(filepath)
        mprint(f"- **{filename}** ({file_size} Bytes)")

In [ ]:
# Test: Persistenz nach "Neustart" (neue Chain-Instanz)
mprint("### Test: Persistenz-Check")
mprint("---")

# Neue Chain-Instanz erstellen (simuliert Neustart)
chain_after_restart = RunnableWithMessageHistory(
    runnable=base_chain,
    get_session_history=get_file_session_history,
    input_messages_key="user_input",
    history_messages_key="history"
)

# Sollte die gespeicherte Historie aus der Datei laden!
response = chain_after_restart.invoke(
    {"user_input": "Kannst du mich noch an meine bisherigen Infos erinnern?"},
    config=config_file
)
mprint(f"**Mensch (nach 'Neustart'):** Kannst du mich noch an meine bisherigen Infos erinnern?")
mprint(f"**KI:** {response}")
mprint("\n✅ **Ergebnis:** Historie wurde erfolgreich aus der Datei geladen!")

## 4.2 | SQLite (Datenbank-Persistenz)

In [ ]:
import sqlite3
from datetime import datetime
from typing import List, Dict, Optional

DB_PATH = "./chat_memory.db"

In [ ]:
class ChatMemoryDB:
    """Einfache Chat-Memory-Datenbank mit SQLite."""

    def __init__(self, db_path: str = DB_PATH):
        self.db_path = db_path
        self._init_db()

    def _init_db(self):
        """Erstellt die Tabellen falls nicht vorhanden."""
        with sqlite3.connect(self.db_path) as conn:
            conn.execute("""
                CREATE TABLE IF NOT EXISTS messages (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    thread_id TEXT NOT NULL,
                    role TEXT NOT NULL,
                    content TEXT NOT NULL,
                    timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
                )
            """)
            conn.execute("""
                CREATE TABLE IF NOT EXISTS summaries (
                    thread_id TEXT PRIMARY KEY,
                    summary TEXT,
                    updated_at DATETIME DEFAULT CURRENT_TIMESTAMP
                )
            """)
            conn.execute("CREATE INDEX IF NOT EXISTS idx_thread ON messages(thread_id)")
            conn.commit()
        print(f"Datenbank initialisiert: {self.db_path}")

    def save_message(self, thread_id: str, role: str, content: str):
        """Speichert eine Nachricht."""
        with sqlite3.connect(self.db_path) as conn:
            conn.execute(
                "INSERT INTO messages (thread_id, role, content) VALUES (?, ?, ?)",
                (thread_id, role, content)
            )
            conn.commit()

    def get_history(self, thread_id: str, limit: Optional[int] = None) -> List[Dict]:
        """Ladt die Historie eines Threads."""
        with sqlite3.connect(self.db_path) as conn:
            if limit:
                rows = conn.execute(
                    "SELECT role, content FROM messages WHERE thread_id = ? ORDER BY id DESC LIMIT ?",
                    (thread_id, limit)
                ).fetchall()
                rows = list(reversed(rows))
            else:
                rows = conn.execute(
                    "SELECT role, content FROM messages WHERE thread_id = ? ORDER BY id",
                    (thread_id,)
                ).fetchall()
        return [{"role": r[0], "content": r[1]} for r in rows]

    def get_message_count(self, thread_id: str) -> int:
        """Zahlt die Nachrichten eines Threads."""
        with sqlite3.connect(self.db_path) as conn:
            count = conn.execute(
                "SELECT COUNT(*) FROM messages WHERE thread_id = ?",
                (thread_id,)
            ).fetchone()[0]
        return count

    def save_summary(self, thread_id: str, summary: str):
        """Speichert/aktualisiert die Zusammenfassung eines Threads."""
        with sqlite3.connect(self.db_path) as conn:
            conn.execute("""
                INSERT OR REPLACE INTO summaries (thread_id, summary, updated_at)
                VALUES (?, ?, CURRENT_TIMESTAMP)
            """, (thread_id, summary))
            conn.commit()

    def get_summary(self, thread_id: str) -> Optional[str]:
        """Ladt die Zusammenfassung eines Threads."""
        with sqlite3.connect(self.db_path) as conn:
            row = conn.execute(
                "SELECT summary FROM summaries WHERE thread_id = ?",
                (thread_id,)
            ).fetchone()
        return row[0] if row else None

    def list_threads(self) -> List[Dict]:
        """Listet alle Threads mit Statistiken."""
        with sqlite3.connect(self.db_path) as conn:
            rows = conn.execute("""
                SELECT thread_id, COUNT(*) as msg_count, MAX(timestamp) as last_msg
                FROM messages
                GROUP BY thread_id
                ORDER BY last_msg DESC
            """).fetchall()
        return [{"thread_id": r[0], "messages": r[1], "last_activity": r[2]} for r in rows]

    def delete_thread(self, thread_id: str):
        """Loscht einen Thread."""
        with sqlite3.connect(self.db_path) as conn:
            conn.execute("DELETE FROM messages WHERE thread_id = ?", (thread_id,))
            conn.execute("DELETE FROM summaries WHERE thread_id = ?", (thread_id,))
            conn.commit()

In [ ]:
class PersistentChatbot:
    """Chatbot mit persistentem Memory uber SQLite."""

    def __init__(self, db_path: str = DB_PATH, max_context: int = 10):
        self.db = ChatMemoryDB(db_path)
        self.max_context = max_context
        self.llm = init_chat_model("openai:gpt-4o-mini", temperature=0.7)
        self.system_prompt = "Du bist ein hilfreicher KI-Assistent mit Gedachtnis."

    def _history_to_messages(self, history: List[Dict]) -> list:
        """Konvertiert DB-Historie zu LangChain-Messages."""
        messages = []
        for item in history:
            if item["role"] == "human":
                messages.append(HumanMessage(content=item["content"]))
            else:
                messages.append(AIMessage(content=item["content"]))
        return messages

    def chat(self, thread_id: str, user_input: str) -> str:
        """Sendet eine Nachricht und speichert die Antwort."""

        # Historie laden (mit Limit)
        history = self.db.get_history(thread_id, limit=self.max_context)
        summary = self.db.get_summary(thread_id)

        # System-Prompt mit Summary erweitern
        enhanced_system = self.system_prompt
        if summary:
            enhanced_system = f"{self.system_prompt}\n\nKontext aus fruheren Gesprachen: {summary}"

        # Messages zusammenbauen
        messages = [SystemMessage(content=enhanced_system)]
        messages.extend(self._history_to_messages(history))
        messages.append(HumanMessage(content=user_input))

        # LLM aufrufen
        response = self.llm.invoke(messages)
        response_text = response.content

        # In DB speichern
        self.db.save_message(thread_id, "human", user_input)
        self.db.save_message(thread_id, "ai", response_text)

        return response_text

    def show_history(self, thread_id: str):
        """Zeigt die Historie eines Threads."""
        history = self.db.get_history(thread_id)
        mprint(f"### Thread: {thread_id} ({len(history)} Nachrichten)")
        mprint("---")
        for i, msg in enumerate(history, 1):
            role = "Human" if msg["role"] == "human" else "KI"
            mprint(f"{i}. **{role}:** {msg['content']}")

    def list_threads(self):
        """Listet alle Threads."""
        threads = self.db.list_threads()
        mprint("### Alle Threads:")
        mprint("---")
        for t in threads:
            mprint(f"- **{t['thread_id']}**: {t['messages']} Nachrichten (zuletzt: {t['last_activity']})")

In [ ]:
# Demo: Persistenter Chatbot
mprint("## SQLite-Chatbot Demo")
mprint("---")

bot = PersistentChatbot()

# Thread 1: Max
print("\n--- Thread: max_session ---")
response = bot.chat("max_session", "Hallo! Ich bin Max und komme aus Munchen.")
mprint(f"**KI:** {response}")

response = bot.chat("max_session", "Ich interessiere mich fur Machine Learning.")
mprint(f"**KI:** {response}")

# Thread 2: Emma
print("\n--- Thread: emma_session ---")
response = bot.chat("emma_session", "Hi! Ich bin Emma aus Berlin.")
mprint(f"**KI:** {response}")

# Zuruck zu Max - Memory bleibt erhalten!
print("\n--- Zuruck zu max_session ---")
response = bot.chat("max_session", "Woher komme ich nochmal?")
mprint(f"**KI:** {response}")

In [ ]:
# Alle Threads anzeigen
bot.list_threads()

In [ ]:
# Historie eines Threads anzeigen
bot.show_history("max_session")

<p><font color='darkblue' size="4">
Test: Neustart-Persistenz
</font></p>

Die Daten bleiben auch nach Neustart erhalten. Fuhren Sie die nachste Zelle aus, um zu testen:

In [ ]:
# Test: Neuer Bot-Instance, gleiche Datenbank
bot2 = PersistentChatbot()

# Sollte Max's Historie kennen!
response = bot2.chat("max_session", "Was war mein Interesse nochmal?")
mprint(f"**KI (nach 'Neustart'):** {response}")

# Historie anzeigen
bot2.show_history("max_session")

# A | Aufgaben
---

<p><font color='black' size="5">
Aufgabe 1: Trimming-Limit testen
</font></p>

**Schwierigkeit:** 1/5

Andern Sie `MAX_MESSAGES` auf 4 und fuhren Sie eine langere Konversation. Beobachten Sie, wann Informationen verloren gehen.

<p><font color='black' size="5">
Aufgabe 2: Summary-Qualitat verbessern
</font></p>

**Schwierigkeit:** 2/5

Verbessern Sie den `summary_prompt`, um wichtige Informationen (Namen, Orte, Praferenzen) besser zu extrahieren.

<p><font color='black' size="5">
Aufgabe 3: Interaktiver CLI-Chatbot
</font></p>

**Schwierigkeit:** 3/5

Erweitern Sie `PersistentChatbot` um eine interaktive Schleife mit Befehlen:
- `exit` - Beenden
- `history` - Historie anzeigen
- `new` - Neuen Thread starten
- `threads` - Alle Threads listen

<p><font color='black' size="5">
Aufgabe 4: Hybrid Memory (Trimming + Summary + DB)
</font></p>

**Schwierigkeit:** 4/5

Kombinieren Sie alle drei Strategien:
1. Speicherung in SQLite
2. Automatisches Trimming auf die letzten N Nachrichten
3. Zusammenfassung der alteren Nachrichten (in DB gespeichert)

# B | Datenbank auslesen
---

Dieser Abschnitt zeigt, wie die SQLite-Datenbank (`chat_memory.db`) direkt ausgelesen werden kann - nutzlich fur Debugging, Analyse oder Export.

In [ ]:
import sqlite3
import os

def read_all_threads_from_db(db_path: str = DB_PATH):
    """
    Liest alle Threads und Nachrichten aus der chat_memory.db Datenbank.
    """
    if not os.path.exists(db_path):
        print(f"Fehler: Datenbankdatei '{db_path}' wurde nicht gefunden.")
        return

    mprint(f"### Lese Datenbank: {db_path}")
    mprint("---")

    with sqlite3.connect(db_path) as conn:
        # Alle Threads mit Statistiken
        threads = conn.execute("""
            SELECT thread_id, COUNT(*) as msg_count, MAX(timestamp) as last_msg
            FROM messages
            GROUP BY thread_id
            ORDER BY last_msg DESC
        """).fetchall()

        if not threads:
            print("Keine Threads in der Datenbank gefunden.")
            return

        mprint(f"**{len(threads)} Threads gefunden**\n")

        # Jeden Thread mit Nachrichten anzeigen
        for thread_id, msg_count, last_msg in threads:
            mprint(f"#### Thread: {thread_id}")
            mprint(f"*{msg_count} Nachrichten, zuletzt: {last_msg}*\n")

            # Nachrichten des Threads
            messages = conn.execute("""
                SELECT role, content, timestamp
                FROM messages
                WHERE thread_id = ?
                ORDER BY id
            """, (thread_id,)).fetchall()

            for i, (role, content, ts) in enumerate(messages, 1):
                role_display = "Human" if role == "human" else "KI"
                # Inhalt kurzen wenn zu lang
                content_short = content[:100] + "..." if len(content) > 100 else content
                mprint(f"{i}. **{role_display}:** {content_short}")

            # Zusammenfassung (falls vorhanden)
            summary = conn.execute(
                "SELECT summary FROM summaries WHERE thread_id = ?",
                (thread_id,)
            ).fetchone()

            if summary and summary[0]:
                mprint(f"\n*Zusammenfassung:* {summary[0][:150]}...")

            mprint("")  # Leerzeile zwischen Threads

In [ ]:
# Alle Threads aus der Datenbank auslesen
read_all_threads_from_db()

In [ ]:
import json

def export_thread_to_json(thread_id: str, db_path: str = DB_PATH) -> dict:
    """Exportiert einen Thread als JSON."""
    with sqlite3.connect(db_path) as conn:
        messages = conn.execute("""
            SELECT role, content, timestamp
            FROM messages
            WHERE thread_id = ?
            ORDER BY id
        """, (thread_id,)).fetchall()

        summary = conn.execute(
            "SELECT summary FROM summaries WHERE thread_id = ?",
            (thread_id,)
        ).fetchone()

    data = {
        "thread_id": thread_id,
        "messages": [
            {"role": r, "content": c, "timestamp": t}
            for r, c, t in messages
        ],
        "summary": summary[0] if summary else None
    }

    return data

# Beispiel: Thread als JSON exportieren
thread_data = export_thread_to_json("max_session")
print(json.dumps(thread_data, indent=2, ensure_ascii=False))

In [ ]:
def delete_all_threads(db_path: str = DB_PATH):
    """Loscht alle Threads aus der Datenbank (Cleanup)."""
    with sqlite3.connect(db_path) as conn:
        conn.execute("DELETE FROM messages")
        conn.execute("DELETE FROM summaries")
        conn.commit()
    print(f"Alle Threads geloscht aus: {db_path}")

# Auskommentiert, um versehentliches Loschen zu verhindern:
# delete_all_threads()